<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Content based image retrieval

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
from google.colab import drive
 
drive.mount('/content/drive')

In [ ]:
ls /content/drive/MyDrive/job\ related/UOC/PDC\ -\ Classificació\ Patrons/Reconeixement\ de\ Patrons/Notebooks/Tema\ 5\:\ Image\ Classification/101_ObjectCategories


In [ ]:
from glob import glob
from os.path import exists, isdir, basename, join, splitext

datasetpath = '/content/drive/My Drive/Docència/Reconeixement de Patrons/Notebooks/Image Classification/101_ObjectCategories/'

cat_paths = [files
              for files in glob(datasetpath + "/*")
              if isdir(files)]
cat_paths.sort()
cats = [basename(cat_path) for cat_path in cat_paths]



We will be using just a number of images and categories from the database to speed-up the execution

In [ ]:
number_of_images_per_class = 20
ncats = 50 #len(cats)

print(cats)
print(ncats)

## Building the dictionary

### Extract features

We will use SIFT features as the previous examples. We encapsulate the code to extract the features of multiple images in the function extractSIFT

In [ ]:
import cv2 as cv

def extractSIFT(input_files):
    """
    Extract SIFT features of a set of images. It returns a dictionary with the features of each image
    """
    all_features_dict = {}
    feature_extractor = cv.SIFT.create()
    for i, fname in enumerate(input_files):
        rgb = cv.cvtColor(cv.imread(fname), cv.COLOR_BGR2RGB)
        gray = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
        kp, desc = feature_extractor.detectAndCompute(gray, None)
        all_features_dict[fname] = desc
    return all_features_dict

Helper function to get all the path of the images:

In [ ]:
EXTENSIONS = [".jpg", ".bmp", ".png", ".pgm", ".tif", ".tiff"]

def get_imgfiles(path):
    """
    get imgfiles returns all the files with the defined extensions under a path
    """
    all_files = []
    all_files.extend([join(path, basename(fname))
                    for fname in glob(path + "/*")
                    if splitext(fname)[-1].lower() in EXTENSIONS])
    return all_files

Once we have defined the helper functions we can extract the features for all the images of the dataset:

In [ ]:
all_files = []
all_files_labels = {}
all_features = {}
cat_label = {}

# for each category
for cat, label in zip(cats, range(ncats)):
    # generate the path for the category:
    cat_path = join(datasetpath, cat)
    # get all the files for the category:
    cat_files = get_imgfiles(cat_path)

    # we keep only a set of the images to speed-up the execution time:
    cat_files = cat_files[:number_of_images_per_class]
    print("label " + str(label) + ": " + cat)

    # extract SIFT features:
    cat_features = extractSIFT(cat_files)
    # store the path for all files
    all_files = all_files + cat_files
    # save the features in the all_features dictionary:
    all_features.update(cat_features)
    # store the label for the category:
    cat_label[cat] = label
    for i in cat_files:
        all_files_labels[i] = label

Lets analyse the features generated:

In [ ]:
print("Number of images in the database = " + str(len(all_features)) + " images")

In [ ]:
# all files has the path of each file, which is used as a key in the dictionary:
file_path = all_files[0]
# print the number of features of each image:
print("First image has " + str(len(all_features[file_path])) + " SIFT descriptors")
print("Each descriptor has " + str(len(all_features[file_path][0])) + " features")


In [ ]:
import numpy as np


count_features = 0
all_features_list = []

for key in all_features:
#  print(key, '-', all_features[key])
  count_features += len(all_features[key])
  # generate a list from all the features
  all_features_list.extend(all_features[key])

print(count_features)
print(len(all_features_list))

# convert dictionary to a list of features:
data = np.array(all_features_list)

print(data.shape)

print("We have " + str(len(all_features)) + " images in the database. ")
print("The total number of descriptors for the images are " + str(count_features) + " (" + str(round(count_features/len(all_features))) +" descriptors per image).")
print("A brute force clustering to find similar images won't work.")

### Cluster the features

Once we have computed all the features we will build a dictionary to encode the features

In [ ]:
# numbers of clusters (words) of our dictionary:
numWords = 200


Next Cell does the cluster of features. It takes several minutes depending on the number of images selected ( 9 minuts with the default configuration)

In [ ]:
from sklearn.cluster import KMeans

import numpy as np

kmeans_features = KMeans(n_clusters=numWords, random_state=0, n_init=5, max_iter=50).fit(data)


### Create an histogram of the features: 

In this step we analyse again all the images: for each feature of each image we assign it to a word

In [ ]:

# Calculate the histogram of features
im_features = np.zeros((len(all_features), numWords), "float32")

for ind, key in enumerate(all_features):

  features_image = all_features[key]

  labels = kmeans_features.predict(features_image)

  # print(len(labels))

  for label in labels: 
    im_features[ind][label] += 1



In [ ]:
im_features

Lets see one histogram generated 

In [ ]:
import matplotlib.pyplot as plt

# image index that we want to see the histogram:
image_number = 117

counts_, bins_, _ = plt.hist(im_features[image_number], bins=im_features[image_number].shape[0],
                             weights=im_features[image_number], range=(0, len(im_features[image_number])))
#plt.show()



We can see the histogram: in the x axis we see the number of the word and in the y axis the occurrences of each word in the image

Finally we will normalize the histogram (to compare images with different number of features):

In [ ]:
from sklearn import preprocessing

im_features = preprocessing.normalize(im_features, norm='l2')


In [ ]:
print(im_features_norm)

With the normalization we have finished the building of the dictionary. From now on we can extract similar images in the database given a query image!! 

### Test the algorithm: Extract similar images to a given example


In [ ]:
# select an image from the database:
# image_path = all_files[95]

# or select a random image from internet:
image_path = "image_query.jpg"
!wget http://www.sedaa.org/wp-content/uploads/2017/04/crocodile-875274_960_720.jpg -O image_query.jpg


In [ ]:

import argparse as ap
import cv2
import imutils 
import numpy as np
import os
# from sklearn.externals import joblib
from scipy.cluster.vq import *

from sklearn import preprocessing
import numpy as np

from pylab import *
from PIL import Image

# we will use the following import to display images in colab:
from google.colab.patches import cv2_imshow

# List where all the descriptors are stored
des_list = []

im = cv2.imread(image_path)
print(image_path)

cv2_imshow(im)


Lets extract the features of the image:

In [ ]:
feature_extractor = cv.SIFT.create()


# read image and extract features:
rgb = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
gray = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
kp, desc = feature_extractor.detectAndCompute(gray, None)


Create the histogram for the features:

In [ ]:

labels = kmeans_features.predict(desc)
# labels = kmeans.predict(desc)


query_image_features = np.zeros((numWords), "float32")

for label in labels: 
  query_image_features[label] += 1

# Perform Tf-Idf vectorization and L2 normalization
# query_image_features = query_image_features*idf
query_image_features = preprocessing.normalize([query_image_features], norm='l2')


Compare the histogram with all the histograms in the database:

In [ ]:

# the histograms are normalized, higher scores means that histograms are similar
score = np.dot(query_image_features, im_features.T)
# argsort sorts the scores from lower to higher. Since we are interested in higher scores we pass the negative scores:
best_matches_sorted = np.argsort(-score)

n_images_to_show = 5

for num in range(n_images_to_show): 

	selected_image = cv.imread(all_files[rank_ID[0][num]])

	cv2_imshow(selected_image)


We are obtaining images from the same category on the database and other images with similar colors and shapes!